In [1]:
"""
Simulation run script for infraSharing5Gupgrade.
Written by Shruthi K A & Ed Oughton.
Jan 2023
"""

import numpy as np
import math
import pandas as pd
import random
import os

params = {
    'MIMO_up': 4,
    'MIMO_down': 4,
    'MUMIMO_number_of_beams': 8,
    'Mode_of_Modulation_and_Code_Rate': 28,
    'modulation_order': 50,
    'TargetcodeRate': 948,
    'Scaling_factor': 1,
    'Mode': 'TDD',
    'Bandwidth_MHz': 10,
    'Frequency_MHz': 700,
    'FrequencySpacing_KHz': 15,
    'overhead': 0.14,
    'Part_of_slots_DL_TDD': 0.7,
    #linkbudgetinputs
    'demand_gb_month': 50,
    'adoption_rate_perc': 50,
    'cell_radius': 10,
    'mimo_layers': 4,
    'beamforming_mmimo': 'yes',
    'mu_mimo_beams': 10,
    'scaling_factor': 1,
    # 'bandwidth_Mhz': 100,
    # 'frequency_Mhz': 3800,
    # 'carrier_spacing_KHz': 15,
    'carrier_aggregation': 1,
    'direction_of_link': 'DL',
    'Max_Tx_Power_dB': 40,
    'number_of_antennas': 16,
    'antenna_Gain_dB': 16,
    'cable_loss_dB': 2,
    'height_gNodeB': 40,
    'type_of_coverage': 'LOS',
    'antennaGain_rx': 1,
    'utnoise_figure': 4,
    'targetSINR': 5,
    'cable_loss_rx': 0,
    'height_UT': 1.5, 
    'coverageLoc': 'Outdoor',
    'number_of_walls':1,
    'depth_of_wall':1,
    'SlowFadingMargin_dB': 7,
    'standarddeviationRural':6,  
    'confidenceInterval': 0.95,
}


def capacity(params):
    #Сalculator allows to calculate the maximum throughput of 5G NR network for user (depending on his mobile device UE) or cell. 
    #Approximately data transfer rate of 5G NR can be calculated using the formula:
    #3GPP specifications TS 38.306 shares a formula for calculating the maximum data rate that a UE/cell can support in downlink and uplink. 
    #v(j)Layers - maximum number of MIMO layers ,3GPP 38.802: maximum 8 in DL, maximum 4 in UL
    #v_up = float(params['MIMO_up']);
    v= float(params['MIMO_down']);
    #nbeam = 1, minimum
    nbeam = float(params['MUMIMO_number_of_beams']);
    #Mode of Modulation and Code Rate:
    MCS = float(params['Mode_of_Modulation_and_Code_Rate']);
    #Q(j)m modulation order (3GPP 38.804, 38.214)
    Qjm = 6;
    #For UL and DL Q(j)m is same (QPSK-2, 16QAM-4, 64QAM-6, 256QAM-8, 1024QAM-10)
    TargetcodeRate = float(params['TargetcodeRate']);                  
    Rmax = TargetcodeRate/1024;    
    #f(j) Scaling factor (3GPP 38.306)
    #1,0.8,0.75,0.4
    f = float(params['Scaling_factor']);
    #BW(j)- band Bandwidth, MHz (3GPP 38.104),
    Bandwidth_MHz = float(params['Bandwidth_MHz']);
    #bw_3800 = float(params['Bandwidth_3800'])
    Mode = params['Mode'];            
    Frequency_MHz = float(params['Frequency_MHz']); 
    #Frequency = float(params['Frequency_mid'])  
    FrequencySpacing_KHz = float(params['FrequencySpacing_KHz']);
    #Overhead OH(j) for control channels Mode
    OH = float(params['overhead']); 
    #FrequencySpacing = float(params'FrequencySpacing');
    #NbwPRB = float(input('Enter PRB (52 for 700 MHz, 10 MHz; 273 for 3800 MHz, 100 MHz)'))
    #NbwPRB, maximum number of PRB (3GPP 38.104) for selected BW(j), FR(j), µ(j).
    NbwPRB = float(math.floor(Bandwidth_MHz*1000/(FrequencySpacing_KHz*12)))-3;
    
    
    if Mode == 'TDD': 
        if Frequency_MHz == 700:
              carrieraggregation = 0;
              #Part of the Slots allocated for DL in TDD mode, 1= 100% slot usage
              T = float(params['Part_of_slots_DL_TDD']); 
        else:
              carrieraggregation = 1;
              #Part of the Slots allocated for DL in FDD mode, 1= 100% slot usage
              T = 1;
    else:
        carrieraggregation = 1;  
        T = 1; #F-OFDM 
    if FrequencySpacing_KHz == 15:
        mu = 0;
        MaxRB = 270;
        if NbwPRB > MaxRB:
            NbwPRB = MaxRB;
    elif FrequencySpacing_KHz == 30:
        mu = 1;
        MaxRB = 273;
        if NbwPRB > MaxRB:
            NbwPRB = MaxRB;
    else:
        mu = 2;
        MaxRB = 135;
        if NbwPRB > MaxRB:
            NbwPRB = MaxRB;
    print(type(NbwPRB),NbwPRB, 'NbwPRB')
    
    # 3GPP 38.101 has specified maximum transmission bandwidth configuration for each UE channel and sub-carrier spacing provided in below table. 
    # The resource block number shown are after removing guard band from channel bandwidth and maximum bandwidth considered is 100 MHz
    GuardBand = float(0.5*Bandwidth_MHz*1000 - (0.5*FrequencySpacing_KHz*12*NbwPRB) - 0.5*FrequencySpacing_KHz);
    print(GuardBand, 'Guard Band (KHz)')
    
    OneRB_size_frequencyDomain_KHz = FrequencySpacing_KHz*12; #In 5G, One NR Resource Block (RB) contains 12 sub-carriers in frequency domain similar to LTE.
    #In LTE resource block bandwidth is fixed to 180 KHz but in NR it is not fixed and depend on sub-carrier spacing.
    OneRB_size_TimeDomain_ms = 1/math.pow(2,mu);
    Ts = 1/(1000*14*math.pow(2,mu))             
    dataRates_Mbps = math.pow(10,-6)*v*Qjm*f*Rmax*NbwPRB*12*(1-OH)/Ts*nbeam;  
    usable_dataRates_Mbps = dataRates_Mbps*T;
    print(OneRB_size_frequencyDomain_KHz, 'OneRB_size_frequencyDomain_KHz')
    print(OneRB_size_TimeDomain_ms, 'OneRB_size_TimeDomain_ms')
    print(Mode, 'Data Rates (Mbps)',dataRates_Mbps)             
    print(Mode, 'Usable Data Rates (Mbps)', usable_dataRates_Mbps)
    
    #To determine a cell’s overall practical capacity for broadband, and thus to evaluate the real capability of any proposed network that leverages wireless
    # technology, one must consider the average of the experience among all users near and far. This is often only 15–25 percent of the theoretical peak for a 
    # single user. When overheads are considered, the usable capacity will typically be less than 75 percent of this value. It therefore is not unusual for 
    # the actual throughput capacity to be only roughly 15 percent of its peak data connection rate – although the latter is the speed that is usually promoted.
    # [Usable_dataRate_Mbps,NbwRB] = capacity(params);
    #print(Usable_dataRate_Mbps)
    ActualCapacity_Mbps = 0.15*usable_dataRates_Mbps;
    print('Actual average Capacity_Mbps', ActualCapacity_Mbps)
    output = []
    filename = "capacityCalculator_{}_{}_{}.csv".format(
            params['Bandwidth_MHz'], 
            params['Frequency_MHz'],
            params['FrequencySpacing_KHz'],
        )
    
    output.append({
        'dataRates_Mbps': dataRates_Mbps, 
        'Bandwidth_MHz':Bandwidth_MHz,
        'Frequency_MHz':Frequency_MHz,
        'FrequencySpacing_KHz':FrequencySpacing_KHz,
        'usable_dataRates_Mbps':usable_dataRates_Mbps,
        'ActualCapacity_Mbps':ActualCapacity_Mbps,
                   })
    output = pd.DataFrame(output)
    if not os.path.exists('results/Capacity'):
        os.mkdir('results/Capacity')
    my_path = os.path.join('results/Capacity', filename)
    output.to_csv(my_path)

def maximumAllowablePathLoss(params):
    fc_MHz = float(params['Frequency_MHz'])
    Bandwidth_MHz = float(params['Bandwidth_MHz'])
    FrequencySpacing_KHz = float(params['FrequencySpacing_KHz'])
    ThermalNoise_dB = -174 + 10*math.log10(Bandwidth_MHz*math.pow(10,6))
    NbwPRB = float(math.floor(Bandwidth_MHz*1000/(FrequencySpacing_KHz*12)))-3;
    if FrequencySpacing_KHz == 15:
        mu = 0;
        MaxRB = 270;
        if NbwPRB > MaxRB:
            NbwPRB = MaxRB;
    elif FrequencySpacing_KHz == 30:
        mu = 1;
        MaxRB = 273;
        if NbwPRB > MaxRB:
            NbwPRB = MaxRB;
    else:
        mu = 2;
        MaxRB = 135;
        if NbwPRB > MaxRB:
            NbwPRB = MaxRB;
    print(type(NbwPRB),NbwPRB, 'NbwPRB')
    SubcarrierQuantity = NbwPRB*12
    #gNodeB configuration
    #Transmit power per PRB, dBm=(Total output power RRH/RRU/AAU) /(Max number of PRB for Bandwidth)
    Max_Tx_Power = float(params['Max_Tx_Power_dB'])
    No_Antennas = float(params['number_of_antennas'])
    Reference_singal_power_dBm = Max_Tx_Power - 10*math.log10(NbwPRB*12)
    Total_transmit_power_dBm = Max_Tx_Power + 10*math.log10(No_Antennas)
    print(Reference_singal_power_dBm, 'Reference_singal_power_dBm')
    print(Total_transmit_power_dBm, 'Total_transmit_power_dBm')
    
    #TransmitPower_TxBW = int(input("Enter total RRH power (in dBm)"))
    TxperPRB = Total_transmit_power_dBm/NbwPRB
    AntennaGain_Tx = float(params['antenna_Gain_dB']);
    CableLoss_Tx = float(params['cable_loss_dB']);
    hBS = float(params['height_gNodeB']);
    c = 3*math.pow(10,8);
    typeCoverage = params['type_of_coverage']
    TypeofCoverageLocation = params['coverageLoc'];
    if TypeofCoverageLocation == 'indoor':
        if fc_MHz == 3800:
            PenetrationLoss_indoor = 25.2;
            walls = int(params['number_of_walls'])
            depth = int(params['depth_of_walls'])
            PenetrationLoss_indoor = PenetrationLoss_indoorperdoor*walls
            AttentuationLoss_indoor = depth*4
        else:
            PenetrationLoss_indoorperdoor = 12.8
            walls = int(params['number_of_walls'])
            depth = int(params['depth_of_walls'])
            PenetrationLoss_indoor = PenetrationLoss_indoorperdoor*walls
            AttentuationLoss_indoor = depth*4
    else:
        PenetrationLoss_indoor = 0;
        AttentuationLoss_indoor = 0;

    direction_of_link = params['direction_of_link']
    #UT configuration
    AntennaGain_Rx = float(params['antennaGain_rx']);
    utnoiseFigure = float(params['utnoise_figure']);
#     QPSk Spectral efficiency: 0.2344 – SINR: -6 dB
# 16QAM Spectral efficiency: 2.5703 -SINR 9 dB
# 64QAM Spectral efficiency: 5.1152 -SINR 21dB
# 256QAM Spectral efficiency: 7.4063 -SINR 35 dB
    TargetSINR = float(params['targetSINR']);
    CableLoss_Rx = float(params['cable_loss_rx']);
    hUT = float(params['height_UT']);
    
    SlowFadingMargin_dB = float(params['SlowFadingMargin_dB']);
    standarddeviationRural = float(params['standarddeviationRural']);
    confidenceInterval = float(params['confidenceInterval']);
    if fc_MHz==3800:
        isd = 5000;
        BodyLoss_dB = 5;
        FoilageLoss_dB = 11;
        RainMargin_dB = 0;
        if direction_of_link == 'DL':
            InterferenceMargain_dB = 6; #calculations for DL
        elif direction_of_link == 'UL':
            InterferenceMargain_dB = 2; #calculations for UL
    else:
        isd = 10000;
        BodyLoss_dB = 2;
        FoilageLoss_dB = 8.5;
        RainMargin_dB = 0;
        InterferenceMargain_dB = 3;
    Receiver_sensitivity_dBm = - utnoiseFigure + ThermalNoise_dB - TargetSINR 
    print(Receiver_sensitivity_dBm)
    # SNRdB = LinkBudget_SignalAtReceiver - ThermalNoise_dB
        
    df1 = []
    #Total_transmit_power_dBm
    max_allowable_prop_pathLoss = Max_Tx_Power - Receiver_sensitivity_dBm + AntennaGain_Tx - CableLoss_Tx + AntennaGain_Rx -\
                                     CableLoss_Rx - FoilageLoss_dB - BodyLoss_dB - InterferenceMargain_dB - \
                                     RainMargin_dB - SlowFadingMargin_dB - PenetrationLoss_indoor - AttentuationLoss_indoor
    print(max_allowable_prop_pathLoss)
    # max_allowable_prop_pathLoss = 87
    lambda_m = c/(fc_MHz*math.pow(10,6))
    print(lambda_m)
    
    
    #invert the free-sapce pathloss 
    constant = 10*math.log10(lambda_m*lambda_m/(16*math.pi*math.pi))
    # cell_radius_m = math.pow(10, (max_allowable_prop_pathLoss+constant)/23)
    # print(cell_radius_m,'cell_radius_m method 1')
    if fc_MHz == 700:
        cell_radius_m = math.pow(10, (max_allowable_prop_pathLoss - 5 - 20*math.log10(fc_MHz))/22)
        print(cell_radius_m,'cell_radius_m method 1')
        isd = 2*cell_radius_m
    elif fc_MHz == 3800:
        cell_radius_m = math.pow(10, (max_allowable_prop_pathLoss/1.05+constant)/22)
        print(cell_radius_m,'cell_radius_m methd 2')
        isd = 3*cell_radius_m
        

    output = []
    filename = "cell_radius_Calculator_{}_{}_{}.csv".format(
            params['Bandwidth_MHz'], 
            params['Frequency_MHz'],
            params['FrequencySpacing_KHz'],
        )
    
    output.append({
        'cell_radius_m': cell_radius_m, 
        'Bandwidth_MHz':Bandwidth_MHz,
        'Frequency_MHz':fc_MHz,
        'FrequencySpacing_KHz':FrequencySpacing_KHz,
        'isd':isd,
                   })
    output = pd.DataFrame(output)
    if not os.path.exists('results/Capacity'):
        os.mkdir('results/Capacity')
    my_path = os.path.join('results/Capacity', filename)
    output.to_csv(my_path)
    
    
    # def linkbudgetEsitimation(params):
#     fc_MHz = float(params['Frequency_MHz'])
#     Bandwidth_MHz = float(params['Bandwidth_MHz'])
#     FrequencySpacing_KHz = float(params['FrequencySpacing_KHz'])
#     NbwPRB = float(math.floor(Bandwidth_MHz*1000/(FrequencySpacing_KHz*12)))-3;
#     if FrequencySpacing_KHz == 15:
#         mu = 0;
#         MaxRB = 270;
#         if NbwPRB > MaxRB:
#             NbwPRB = MaxRB;
#     elif FrequencySpacing_KHz == 30:
#         mu = 1;
#         MaxRB = 273;
#         if NbwPRB > MaxRB:
#             NbwPRB = MaxRB;
#     else:
#         mu = 2;
#         MaxRB = 135;
#         if NbwPRB > MaxRB:
#             NbwPRB = MaxRB;
#     print(type(NbwPRB),NbwPRB, 'NbwPRB')
#     SubcarrierQuantity = NbwPRB*12
#     #gNodeB configuration
#     #Transmit power per PRB, dBm=(Total output power RRH/RRU/AAU) /(Max number of PRB for Bandwidth)
#     Max_Tx_Power = float(params['Max_Tx_Power_dB'])
#     No_Antennas = float(params['number_of_antennas'])
#     Reference_singal_power_dBm = Max_Tx_Power - 10*math.log10(NbwPRB*12)
#     Total_transmit_power_dBm = Max_Tx_Power + 10*math.log10(No_Antennas)
#     print(Reference_singal_power_dBm, 'Reference_singal_power_dBm')
#     print(Total_transmit_power_dBm, 'Total_transmit_power_dBm')
    
#     #TransmitPower_TxBW = int(input("Enter total RRH power (in dBm)"))
#     TxperPRB = Total_transmit_power_dBm/NbwPRB
#     AntennaGain_Tx = float(params['antenna_Gain_dB']);
#     CableLoss_Tx = float(params['cable_loss_dB']);
#     hBS = float(params['height_gNodeB']);
#     c = 3*math.pow(10,8);
#     typeCoverage = params['type_of_coverage']
#     direction_of_link = params['direction_of_link']
#     #UT configuration
#     AntennaGain_Rx = float(params['antennaGain_rx']);
#     utnoiseFigure = float(params['utnoise_figure']);
# #     QPSk Spectral efficiency: 0.2344 – SINR: -6 dB
# # 16QAM Spectral efficiency: 2.5703 -SINR 9 dB
# # 64QAM Spectral efficiency: 5.1152 -SINR 21dB
# # 256QAM Spectral efficiency: 7.4063 -SINR 35 dB
#     TargetSINR = float(params['targetSINR']);
#     CableLoss_Rx = float(params['cable_loss_rx']);
#     hUT = float(params['height_UT']);
#     TypeofCoverage = params['coverageType'];
#     if TypeofCoverage == 'indoor':
#         if fc_MHz == 3800:
#             PenetrationLoss_indoor = 25.2;
#             walls = int(params['number_of_walls'])
#             depth = int(params['depth_of_walls'])
#             PenetrationLoss_indoor = PenetrationLoss_indoorperdoor*walls
#             AttentuationLoss_indoor = depth*4
#         else:
#             PenetrationLoss_indoorperdoor = 12.8
#             walls = int(params['number_of_walls'])
#             depth = int(params['depth_of_walls'])
#             PenetrationLoss_indoor = PenetrationLoss_indoorperdoor*walls
#             AttentuationLoss_indoor = depth*4
#     else:
#         PenetrationLoss_indoor = 0;
#         AttentuationLoss_indoor = 0;

#     #AdditionalLosses
#     SlowFadingMargin_dB = float(params['SlowFadingMargin_dB']);
#     standarddeviationRural = float(params['standarddeviationRural']);
#     confidenceInterval = float(params['confidenceInterval']);
#     if fc_MHz==3800:
#         isd = 5000;
#         BodyLoss_dB = 5;
#         FoilageLoss_dB = 11;
#         RainMargin_dB = 0;
#         if direction_of_link == 'DL':
#             InterferenceMargain_dB = 6; #calculations for DL
#         elif direction_of_link == 'UL':
#             InterferenceMargain_dB = 2; #calculations for UL
#     else:
#         isd = 10000;
#         BodyLoss_dB = 2;
#         FoilageLoss_dB = 8.5;
#         RainMargin_dB = 0;
#         InterferenceMargain_dB = 3;
        
#     df1 = []
#     for d_UE_Tx in range(100,10000,300):

#         output = []
#         for _ in range(20):
#             PathLoss_PropagationModel = propgationLosses(hBS,hUT,fc_MHz,c,d_UE_Tx,typeCoverage,confidenceInterval);
#             LinkBudget_SignalAtReceiver = Total_transmit_power_dBm + AntennaGain_Tx - CableLoss_Tx + AntennaGain_Rx -\
#                                             CableLoss_Rx - PathLoss_PropagationModel - FoilageLoss_dB - BodyLoss_dB - InterferenceMargain_dB - \
#                                             RainMargin_dB - SlowFadingMargin_dB - PenetrationLoss_indoor - AttentuationLoss_indoor
#             ThermalNoise_dB = -174 + 10*math.log10(Bandwidth_MHz*math.pow(10,6))
#             Receiver_sensitivity_dBm = - utnoiseFigure + ThermalNoise_dB - TargetSINR + AntennaGain_Rx
#             SNRdB = LinkBudget_SignalAtReceiver - ThermalNoise_dB
#             SNR =math.pow(10,(SNRdB/10))
#             SNR = (1/d_UE_Tx)*LinkBudget_SignalAtReceiver/ThermalNoise_dB
#             if abs(Receiver_sensitivity_dBm) > abs(LinkBudget_SignalAtReceiver):
#                 y = "pass";
#                 channelThrouput=Bandwidth_MHz*math.pow(10,6)*math.log10(1+SNR)
#             else:
#                 y = "fail"
#                 channelThrouput = 0;
#             channelThroughout_Mbps = channelThrouput/math.pow(10,6);
#             T = float(params['Part_of_slots_DL_TDD']);
#             spectralefficiency = channelThrouput/(Bandwidth_MHz*math.pow(10,6)*T)
#             output.append({
#                 'iteration': _,
#                 'bandwidth': Bandwidth_MHz,
#                 'frequecy': fc_MHz,
#                 'confidence_interval':confidenceInterval,
#                 'distance_m': d_UE_Tx,
#                 'inter_cell_site_distance': isd,
#                 'path_loss_db': PathLoss_PropagationModel,
#                 'LinkBudget_SignalAtReceiver':LinkBudget_SignalAtReceiver,
#                 'sinr': SNRdB,
#                 'receiverSensitivty': Receiver_sensitivity_dBm,
#                 'radio_channel_status': y,
#                 'spectral_efficiency': spectralefficiency,
#                 'capacity_Mbps': channelThroughout_Mbps,
#                 'type':typeCoverage,
#             })
       
        
#             output = pd.DataFrame(output)
#             df1 = pd.DataFrame(df1)
#             if _ == 1:
#                 df1.drop(df1.index, inplace=True,index=False)
#                 df1 = output
#             else:
#                 df1 = df1.append(output,ignore_index=True)

#             filename = "LinkBudget{}_{}_{}_{}.csv".format(
#                 params['Frequency_MHz'], 
#                 params['Bandwidth_MHz'], 
#                 params['FrequencySpacing_KHz'],
#                 params['type_of_coverage'],
#             )
#             if not os.path.exists('results/Capacity'):
#                 os.mkdir('results/Capacity')
#             my_path = os.path.join('results/Capacity', filename)
#             df1.to_csv(my_path, index=False)   

        
# def pathlossLOScal(d3D,fc_MHz,h,x):
#     PLRMaLOS = 20*math.log10(40*math.pi*d3D*fc_MHz/3000) + min(0.03*math.pow(h,1.72), 10)*math.log10(d3D) - min(0.044*math.pow(h,1.72), 14.77) + 0.002*math.log10(h)*d3D
#     return PLRMaLOS

# #model propagation Losses
# def propgationLosses(hBS,hUT,fc,c,d_UE_Tx,typeCoverage,confidenceInterval):
#     dBP = 2*math.pi*hBS*hUT*fc*math.pow(10,6)/c;
#     d3D = math.sqrt(math.pow(d_UE_Tx,2)+math.pow(hBS-hUT,2))
#     print(d3D/dBP)
#     W = 20; #m average street width
#     h = 5; #m average building height
#     if confidenceInterval == 0.95:
#         z = 1.959;
#     elif confidenceInterval == 0.9:
#         z = 1.644;
#     elif confidenceInterval == 0.8:
#         z = 1.28;
#     elif confidenceInterval == 0.05:
#         z = 0.06;
#     if fc == 700:
#         if typeCoverage == "LOS":
#             standarddeviationRural = 5.6;
#             mu = 0;
#             #PL_CIH_3GPP_LOS = 32.4 + 20*math.log10(fc/1000) +23.1*(1-0.006*(hBS-35)/35)*math.log10(d_UE_Tx) + mu + z*standarddeviationRural;
#             PL_CIH_3GPP_LOS = 32.4 + 20*math.log10(fc/1000) +23.1*(1-0.003*(hBS-35)/35)*math.log10(d_UE_Tx) + mu + z*standarddeviationRural;
#             PLRMaLOSresults = PL_CIH_3GPP_LOS;
#         elif typeCoverage == "NLOS":
#             standarddeviationRural = 8.7
#             mu = 0;
#             #29.4 - first term
#             #PL_CIH_3GPP_NLOS = 32.4 + 20*math.log10(fc/1000) +23.1*(1-0.006*(hBS-35)/35)*math.log10(d_UE_Tx) + mu + z*standarddeviationRural;
#             PL_CIH_3GPP_NLOS = 32.4 + 20*math.log10(fc/1000) +23.1*(1-0.006*(hBS-35)/35)*math.log10(d_UE_Tx) + mu + z*standarddeviationRural;
#             PLRMaLOSresults = PL_CIH_3GPP_NLOS
#     elif fc == 3800:
#         if typeCoverage == "LOS":
#             standarddeviationRural = 5.6;
#             mu = 0;
#             #44.03 - first term
#             #PL_CIH_3GPP_LOS = 32.4 + 20*math.log10(fc/1000) +23.1*(1-0.003*(hBS-35)/35)*math.log10(d_UE_Tx) + mu + z*standarddeviationRural;
#             PL_CIH_3GPP_LOS = 32.4 + 20*math.log10(fc/1000) +30.7*(1-0.06*(hBS-35)/35)*math.log10(d_UE_Tx) + mu + z*standarddeviationRural;
#             PLRMaLOSresults = PL_CIH_3GPP_LOS;
#         elif typeCoverage == "NLOS":
#             standarddeviationRural = 8.7
#             mu = 0;
#             #PL_CIH_3GPP_NLOS = 32.4 + 20*math.log10(fc/1000) +30.7*(1-0.06*(hBS-35)/35)*math.log10(d_UE_Tx) + mu + z*standarddeviationRural;
#             PL_CIH_3GPP_NLOS = 32.4 + 20*math.log10(fc/1000) +30.7*(1-0.06*(hBS-35)/35)*math.log10(d_UE_Tx) + mu + z*standarddeviationRural;
#             PLRMaLOSresults = PL_CIH_3GPP_NLOS
    
#     #  # CIH RMa path loss models
#     # if typeCoverage == "LOS":
#     #         standarddeviationRural = 1.7;
#     #         mu = 0;
#     #         PL_CIH_3GPP_LOS = 32.4 + 20*math.log10(fc/1000) +23.1*(1-0.03*(hBS-35)/35)*math.log10(d_UE_Tx) + mu + z*standarddeviationRural;
#     #         PLRMaLOSresults = PL_CIH_3GPP_LOS;
#     # elif typeCoverage == "NLOS":
#     #         standarddeviationRural = 6.7
#     #         mu = 0;
#     #         #29.4 - first term
#     #         #PL_CIH_3GPP_NLOS = 32.4 + 20*math.log10(fc/1000) +23.1*(1-0.006*(hBS-35)/35)*math.log10(d_UE_Tx) + mu + z*standarddeviationRural;
#     #         PL_CIH_3GPP_NLOS = 32.4 + 20*math.log10(fc/1000) +23.1*(1-0.006*(hBS-35)/35)*math.log10(d_UE_Tx) + mu + z*standarddeviationRural;
#     #         PLRMaLOSresults = PL_CIH_3GPP_NLOS
#     return PLRMaLOSresults

a = np.array([1,2])

for a in a:
    capacity(params)
    maximumAllowablePathLoss(params)
    if a == 2:
        params['Frequency_MHz'] = 3800
        params['Bandwidth_MHz'] = 100        
        capacity(params)
        maximumAllowablePathLoss(params)
# NbwPRB = practicalCapacityBroadband()




<class 'float'> 52.0 NbwPRB
312.5 Guard Band (KHz)
180.0 OneRB_size_frequencyDomain_KHz
1.0 OneRB_size_TimeDomain_ms
TDD Data Rates (Mbps) 1335.42864
TDD Usable Data Rates (Mbps) 934.800048
Actual average Capacity_Mbps 140.2200072
<class 'float'> 52.0 NbwPRB
12.048154103175758 Reference_singal_power_dBm
52.04119982655925 Total_transmit_power_dBm
-113.0
147.5
0.42857142857142855
7777.14083549656 cell_radius_m method 1
<class 'float'> 52.0 NbwPRB
312.5 Guard Band (KHz)
180.0 OneRB_size_frequencyDomain_KHz
1.0 OneRB_size_TimeDomain_ms
TDD Data Rates (Mbps) 1335.42864
TDD Usable Data Rates (Mbps) 934.800048
Actual average Capacity_Mbps 140.2200072
<class 'float'> 52.0 NbwPRB
12.048154103175758 Reference_singal_power_dBm
52.04119982655925 Total_transmit_power_dBm
-113.0
147.5
0.42857142857142855
7777.14083549656 cell_radius_m method 1
<class 'int'> 270 NbwPRB
25692.5 Guard Band (KHz)
180.0 OneRB_size_frequencyDomain_KHz
1.0 OneRB_size_TimeDomain_ms
TDD Data Rates (Mbps) 6933.9564
TDD Usable

In [111]:
# print(params['Frequency_MHz'])